# Analyse des différences Python vs R (IQL v7)
# 
## Ce notebook permet d'explorer les cas de divergence entre les résultats Python et R.


### 1. Chargement des données


In [ ]:
import pandas as pd
import numpy as np

# Charger le fichier de validation
df = pd.read_excel("val_py0r7.xlsx", sheet_name=0)

print(f"Total séjours: {len(df)}")
print(f"Colonnes: {len(df.columns)}")

Total séjours: 2389
Colonnes: 58


### 2. Vue d'ensemble des concordances/discordances


In [2]:
# Statut de concordance
print("=== CONCORDANCE GLOBALE ===\n")
print(df["pyr_st"].value_counts(dropna=False))

# Méthode robuste : convertir en string puis comparer
concordants = df["pyr_st"].astype(str).str.upper().eq("TRUE").sum()
discordants = df["pyr_st"].astype(str).str.upper().eq("FALSE").sum()
total = concordants + discordants

print(f"\nConcordants: {concordants}")
print(f"Discordants: {discordants}")

if total > 0:
    print(f"\nTaux de concordance: {concordants / total * 100:.1f}%")
    print(f"Taux de discordance: {discordants / total * 100:.1f}%")


=== CONCORDANCE GLOBALE ===

pyr_st
True     2340
False      49
Name: count, dtype: int64

Concordants: 2340
Discordants: 49

Taux de concordance: 97.9%
Taux de discordance: 2.1%


In [3]:
# Matrice de confusion des classes
print("=== MATRICE DE CONFUSION (Python x R) ===\n")
confusion = pd.crosstab(
    df["sej_classe.x"].fillna("NA"),
    df["sej_classe.y"].fillna("NA"),
    margins=True,
    margins_name="Total",
)
print(confusion)

=== MATRICE DE CONFUSION (Python x R) ===



KeyError: 'sej_classe.x'

### 3. Documents non retrouvés par Python 
#### Python n'a trouvé aucun document alors que R en a trouvé un.


In [ ]:
# Filtrer les cas où Python = sansLL et R = LL, ET Python n'a pas de doc_id
docs_non_trouves = df[
    (df["sej_classe.x"] == "sansLL")
    & (df["sej_classe.y"].isin(["0j", "1j+"]))
    & (df["doc_id.x"].isna())
].copy()

print(f"=== DOCUMENTS NON RETROUVÉS PAR PYTHON ===")
print(f"Nombre de cas: {len(docs_non_trouves)}")


=== DOCUMENTS NON RETROUVÉS PAR PYTHON ===
Nombre de cas: 89


#### 3.1 🎯 CAUSE A : Borne de date exclusive (31/01 exclu)


In [ ]:
# Analyser la distribution des dates de validation
print("=== DISTRIBUTION DES DATES DE VALIDATION ===\n")

# Docs trouvés par Python
cas_trouves = df[df["doc_id.x"].notna()]
print(f"Docs trouvés par Python: {len(cas_trouves)}")
print("\nDistribution doc_val.x (derniers jours):")
print(cas_trouves["doc_val.x"].value_counts().sort_index().tail(12))

# Vérifier le 31/01
docs_31_jan_py = cas_trouves[cas_trouves["doc_val.x"] == "2025-01-31"]
print(f"\n🎯 Docs Python validés le 31/01/2025: {len(docs_31_jan_py)}")


=== DISTRIBUTION DES DATES DE VALIDATION ===

Docs trouvés par Python: 2257

Distribution doc_val.x (derniers jours):
doc_val.x
2025-01-19     43
2025-01-20     84
2025-01-21     93
2025-01-22     97
2025-01-23    103
2025-01-24    103
2025-01-25     60
2025-01-26     54
2025-01-27     83
2025-01-28     97
2025-01-29     82
2025-01-30     99
Name: count, dtype: int64

🎯 Docs Python validés le 31/01/2025: 0


In [ ]:
# Docs R du 31/01 non trouvés par Python
docs_non_trouves["doc_val_date"] = pd.to_datetime(docs_non_trouves["doc_val.y"]).dt.date

print("=== DATES DE VALIDATION DES DOCS R NON TROUVÉS ===\n")
print(docs_non_trouves["doc_val.y"].value_counts().sort_index())

# Séparer par date
cause_a_31jan = docs_non_trouves[docs_non_trouves["doc_val.y"] == "2025-01-31"]
cause_a_apres = docs_non_trouves[docs_non_trouves["doc_val.y"] > "2025-01-31"]

print(f"\n📊 Répartition:")
print(f"  - Doc validé le 31/01/2025 (BORNE EXCLUE):  {len(cause_a_31jan)} cas")
print(f"  - Doc validé APRÈS 31/01/2025 (hors période): {len(cause_a_apres)} cas")


=== DATES DE VALIDATION DES DOCS R NON TROUVÉS ===

doc_val.y
2025-01-31    53
2025-02-03     1
2025-02-04     2
2025-02-06     6
2025-02-07     2
2025-02-08     6
2025-02-11     1
2025-02-13     2
2025-02-14     5
2025-02-16     2
2025-02-17     1
2025-02-18     2
2025-02-19     1
2025-03-06     1
2025-04-25     1
2025-05-07     2
2025-09-22     1
Name: count, dtype: int64

📊 Répartition:
  - Doc validé le 31/01/2025 (BORNE EXCLUE):  53 cas
  - Doc validé APRÈS 31/01/2025 (hors période): 36 cas


#### 3.2 CAUSE B : Documents validés après la période demandée

In [ ]:
print("=" * 60)
print("CAUSE B : DOCUMENTS VALIDÉS APRÈS LA PÉRIODE (36 cas)")
print("=" * 60)

print(f"\nNombre de cas: {len(cause_a_apres)}")

# Analyser ces cas
print("\nDistribution des dates de validation (après période):")
print(cause_a_apres["doc_val.y"].value_counts().sort_index().head(10))

# Vérifier si doc_venue correspond
avec_venue = cause_a_apres[cause_a_apres["doc_venue.y"].notna()]
sans_venue = cause_a_apres[cause_a_apres["doc_venue.y"].isna()]
print(f"\n  - R a trouvé via doc_venue: {len(avec_venue)}")
print(f"  - R a trouvé sans doc_venue: {len(sans_venue)}")


CAUSE B : DOCUMENTS VALIDÉS APRÈS LA PÉRIODE (36 cas)

Nombre de cas: 36

Distribution des dates de validation (après période):
doc_val.y
2025-02-03    1
2025-02-04    2
2025-02-06    6
2025-02-07    2
2025-02-08    6
2025-02-11    1
2025-02-13    2
2025-02-14    5
2025-02-16    2
2025-02-17    1
Name: count, dtype: int64

  - R a trouvé via doc_venue: 30
  - R a trouvé sans doc_venue: 6


### 4. CAUSE C : Doc Python sans spécialité (57 cas)
 
#### Python a trouvé un document mais la jointure avec la matrice n'a pas donné de spécialité, alors que R a trouvé un AUTRE document avec spécialité.


In [ ]:
# Filtrer les cas où Python a un doc mais pas de spécialité
cause_c = df[
    (df["sej_classe.x"] == "sansLL")
    & (df["sej_classe.y"].isin(["0j", "1j+"]))
    & (df["doc_id.x"].notna())
    & (df["sej_spe.x"].isna())
].copy()

print("=" * 60)
print("🎯 CAUSE C : DOC PYTHON SANS SPÉCIALITÉ (57 cas)")
print("=" * 60)

print(f"\nNombre de cas: {len(cause_c)}")
print("""
PROBLÈME IDENTIFIÉ:
- Python trouve un doc (ex: "CR Urgences") mais sans spécialité dans la matrice
- R trouve un AUTRE doc (ex: "CR LL Cardiologie") avec spécialité
- La jointure matrice est faite APRÈS le tri en Python, AVANT en R

SOLUTION (src/data_processing.py):
  Faire la jointure matrice AVANT le tri, comme en R (ligne 184)
""")


🎯 CAUSE C : DOC PYTHON SANS SPÉCIALITÉ (57 cas)

Nombre de cas: 57

PROBLÈME IDENTIFIÉ:
- Python trouve un doc (ex: "CR Urgences") mais sans spécialité dans la matrice
- R trouve un AUTRE doc (ex: "CR LL Cardiologie") avec spécialité
- La jointure matrice est faite APRÈS le tri en Python, AVANT en R

SOLUTION (src/data_processing.py):
  Faire la jointure matrice AVANT le tri, comme en R (ligne 184)



In [ ]:
# Types de documents Python sans mapping
print("\n=== Types de documents Python sans mapping ===\n")
print(cause_c["doc_libelle.x"].value_counts())

# %%
# Comparaison doc Python vs doc R
print("\n=== Comparaison Python vs R ===\n")
cols_c = [
    "sej_id",
    "sej_uf.x",
    "doc_libelle.x",
    "sej_spe.x",
    "doc_libelle.y",
    "sej_spe.y",
]
print(cause_c[cols_c].head(15).to_string())



=== Types de documents Python sans mapping ===

doc_libelle.x
CR Urgences                                                  42
CR HDJ Oncologie Foch                                         4
CR Lettre de Liaison Réa Foch                                 4
CR Lettre de Liaison Pneumologie Foch                         2
CR Lettre de Liaison Médecine interne et Polyvalente Foch     1
CR Lettre de Liaison Médecine Interne Foch                    1
CR Lettre de Liaison Oncologie HDJ ILR Foch                   1
CR Lettre de Liaison Cardiologie Foch                         1
CR Lettre de Liaison HDJ Education Diabétologie Foch          1
Name: count, dtype: int64

=== Comparaison Python vs R ===

         sej_id  sej_uf.x                                              doc_libelle.x sej_spe.x                                   doc_libelle.y         sej_spe.y
2     240281460       338                                      CR HDJ Oncologie Foch       NaN     CR Lettre de Liaison Unité Vanderbilt Foc

### 5. CAUSE D : Documents différents sélectionnés (21 cas avec spécialité)
 
#### Python et R ont tous deux trouvé un document avec spécialité, mais pas le même.


In [ ]:
# Filtrer les cas où doc_id diffère ET Python a une spécialité
cause_d = df[
    (df["doc_id.x"] != df["doc_id.y"])
    & (df["doc_id.x"].notna())
    & (df["doc_id.y"].notna())
    & (df["sej_spe.x"].notna())
].copy()

print("=" * 60)
print("CAUSE D : DOCUMENTS DIFFÉRENTS SÉLECTIONNÉS (21 cas)")
print("=" * 60)

print(f"\nNombre de cas: {len(cause_d)}")


CAUSE D : DOCUMENTS DIFFÉRENTS SÉLECTIONNÉS (21 cas)

Nombre de cas: 21


In [ ]:
print("\n=== Comparaison des critères de sélection ===\n")
print(f"{'Critère':<12} | {'Python True':>12} | {'R True':>12}")
print("-" * 42)
print(
    f"{'sdt_docven':<12} | {cause_d['sdt_docven.x'].sum():>12} | {cause_d['sdt_docven.y'].sum():>12}"
)
print(
    f"{'sdt_docval':<12} | {cause_d['sdt_docval.x'].sum():>12} | {cause_d['sdt_docval.y'].sum():>12}"
)
print(
    f"{'sdt_smere':<12} | {cause_d['sdt_smere.x'].sum():>12} | {cause_d['sdt_smere.y'].sum():>12}"
)
print(
    f"{'sdt_doccre':<12} | {cause_d['sdt_doccre.x'].sum():>12} | {cause_d['sdt_doccre.y'].sum():>12}"
)
print(
    f"{'sdt_doccref':<12} | {cause_d['sdt_doccref.x'].sum():>12} | {cause_d['sdt_doccref.y'].sum():>12}"
)
print(
    f"{'sdt_emere':<12} | {cause_d['sdt_emere.x'].sum():>12} | {cause_d['sdt_emere.y'].sum():>12}"
)
print(
    f"{'sdt_status':<12} | {cause_d['sdt_status.x'].sum():>12} | {cause_d['sdt_status.y'].sum():>12}"
)

# %%
# Exemples
print("\n=== Exemples docs différents ===\n")
cols_d = [
    "sej_id",
    "sej_uf.x",
    "doc_id.x",
    "sej_spe.x",
    "del_sorval.x",
    "sdt_docven.x",
    "doc_id.y",
    "sej_spe.y",
    "del_sorval.y",
    "sdt_docven.y",
]
print(cause_d[cols_d].head(10).to_string())


=== Comparaison des critères de sélection ===

Critère      |  Python True |       R True
------------------------------------------
sdt_docven   |            0 |           20
sdt_docval   |            8 |           20
sdt_smere    |           17 |           19
sdt_doccre   |           12 |           21
sdt_doccref  |            3 |           14
sdt_emere    |           14 |           21
sdt_status   |            4 |           18

=== Exemples docs différents ===

         sej_id  sej_uf.x    doc_id.x         sej_spe.x  del_sorval.x  sdt_docven.x    doc_id.y         sej_spe.y  del_sorval.y sdt_docven.y
344   250000418       396  39111841.0          UROLOGIE           NaN         False  39276570.0          UROLOGIE          31.0         True
615   250002688       396  38945580.0          UROLOGIE          13.0         False  39169683.0          UROLOGIE          24.0         True
700   250004795       330  38862568.0       PNEUMOLOGIE          -1.0         False  39752405.0       PNEUM

### 6. CAUSE E : Python=LL mais R=sansLL (8 cas)


In [ ]:
cause_e = df[
    (df["sej_classe.x"].isin(["0j", "1j+"])) & (df["sej_classe.y"] == "sansLL")
].copy()

print("=" * 60)
print("CAUSE E : PYTHON=LL MAIS R=SANSLL (8 cas)")
print("=" * 60)

print(f"\nNombre de cas: {len(cause_e)}")

if len(cause_e) > 0:
    cols_e = [
        "sej_id",
        "sej_uf.x",
        "doc_id.x",
        "doc_libelle.x",
        "sej_spe.x",
        "del_val.x",
        "sej_classe.x",
        "doc_id.y",
        "sej_spe.y",
        "del_val.y",
        "sej_classe.y",
    ]
    print(cause_e[cols_e].to_string())

CAUSE E : PYTHON=LL MAIS R=SANSLL (8 cas)

Nombre de cas: 8
         sej_id  sej_uf.x    doc_id.x                               doc_libelle.x         sej_spe.x  del_val.x sej_classe.x    doc_id.y         sej_spe.y  del_val.y sej_classe.y
700   250004795       330  38862568.0       CR Lettre de Liaison Pneumologie Foch       PNEUMOLOGIE        0.0           0j  39752405.0       PNEUMOLOGIE        NaN       sansLL
833   250009057       456  38967293.0       CR Lettre de Liaison Pneumologie Foch       PNEUMOLOGIE        0.0           0j  38967293.0       PNEUMOLOGIE        NaN       sansLL
1457  259000120       324  38995702.0         CR Lettre de Liaison Oncologie Foch         ONCOLOGIE        0.0           0j  39137232.0         ONCOLOGIE        NaN       sansLL
1490  259000313       390  39022099.0               CR Lettre de Liaison ORL Foch               ORL        0.0           0j  39022099.0               ORL        NaN       sansLL
1722  259001483       336  39154836.0  CR Lettre d

## 7. Résumé des causes

In [ ]:
print("=" * 70)
print("📊 RÉSUMÉ DES CAUSES DE DIVERGENCE")
print("=" * 70)

# Recalculer les totaux
total_discordants = discordants

print(f"\nTotal discordants: {total_discordants}\n")
print(f"{'Cause':<45} | {'Nb':>5} | {'%':>6}")
print("-" * 62)
print(
    f"{'A. Borne date exclusive (31/01 exclu)':<45} | {len(cause_a_31jan):>5} | {len(cause_a_31jan) / total_discordants * 100:>5.1f}%"
)
print(
    f"{'B. Docs validés après période':<45} | {len(cause_a_apres):>5} | {len(cause_a_apres) / total_discordants * 100:>5.1f}%"
)
print(
    f"{'C. Doc Python sans spécialité (tri)':<45} | {len(cause_c):>5} | {len(cause_c) / total_discordants * 100:>5.1f}%"
)
print(
    f"{'D. Docs différents (autres critères)':<45} | {len(cause_d):>5} | {len(cause_d) / total_discordants * 100:>5.1f}%"
)
print(
    f"{'E. Python=LL, R=sansLL':<45} | {len(cause_e):>5} | {len(cause_e) / total_discordants * 100:>5.1f}%"
)
print("-" * 62)

📊 RÉSUMÉ DES CAUSES DE DIVERGENCE

Total discordants: 175

Cause                                         |    Nb |      %
--------------------------------------------------------------
A. Borne date exclusive (31/01 exclu)         |    53 |  30.3%
B. Docs validés après période                 |    36 |  20.6%
C. Doc Python sans spécialité (tri)           |    57 |  32.6%
D. Docs différents (autres critères)          |    21 |  12.0%
E. Python=LL, R=sansLL                        |     8 |   4.6%
--------------------------------------------------------------


In [ ]:
total_explique = (
    len(cause_a_31jan) + len(cause_a_apres) + len(cause_c) + len(cause_d) + len(cause_e)
)
print(
    f"{'TOTAL EXPLIQUÉ':<45} | {total_explique:>5} | {total_explique / total_discordants * 100:>5.1f}%"
)

non_explique = total_discordants - total_explique
if non_explique > 0:
    print(
        f"{'Non expliqué':<45} | {non_explique:>5} | {non_explique / total_discordants * 100:>5.1f}%"
    )


TOTAL EXPLIQUÉ                                |   175 | 100.0%


### 8. Export des cas pour analyse manuelle

In [ ]:
with pd.ExcelWriter("analyse_causes_divergence.xlsx", engine="openpyxl") as writer:
    cause_a_31jan.to_excel(writer, sheet_name="CauseA_Borne31jan", index=False)
    cause_a_apres.to_excel(writer, sheet_name="CauseB_ApresPerdiode", index=False)
    cause_c.to_excel(writer, sheet_name="CauseC_DocSansSpe", index=False)
    cause_d.to_excel(writer, sheet_name="CauseD_DocDifferent", index=False)
    if len(cause_e) > 0:
        cause_e.to_excel(writer, sheet_name="CauseE_PyLL_RsansLL", index=False)

print("✅ Fichier 'analyse_causes_divergence.xlsx' créé avec les cas par cause")


✅ Fichier 'analyse_causes_divergence.xlsx' créé avec les cas par cause
